In [502]:
# Cell 1: Getting started

import pandas as pd
import altair as alt
import numpy as np
from vega_datasets import data

# Loading up the main dataset
df = pd.read_csv('country_economics_data.csv')

# I'm stripping out the noise and keeping only the 5 columns I actually need for the visuals
# Also renaming 'ID' so I remember it's the ISO code later on
required_columns = ['Name', 'ID', 'Region', 'GDP', 'Population']
df = df[required_columns].copy()

# quick sanity check to make sure the data looks right before I start math
print("Dataset Loaded Successfully.")
print(f"Total Countries: {len(df)}")
print("-" * 30)
print(df.head())
print("-" * 30)
print(df.info())

Dataset Loaded Successfully.
Total Countries: 173
------------------------------
          Name  ID    Region  GDP  Population
0  Afghanistan   4      Asia   17       35.70
1      Albania   8    Europe   27        2.36
2      Algeria  12    Africa  264       46.81
3       Angola  24    Africa   80       35.12
4    Argentina  32  Americas  633       47.07
------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Name        173 non-null    object 
 1   ID          173 non-null    int64  
 2   Region      173 non-null    object 
 3   GDP         173 non-null    int64  
 4   Population  173 non-null    float64
dtypes: float64(1), int64(2), object(2)
memory usage: 6.9+ KB
None


In [503]:
# Cell 2: Doing the math & Exporting the Data

# 1. Global Totals
# First, I need the world totals to calculate shares
total_gdp = df['GDP'].sum()
total_pop = df['Population'].sum()

# 2. Shares
# Calculating exactly how much of the global pie each country holds
df['% Global GDP'] = (df['GDP'] / total_gdp) * 100
df['% Global Pop'] = (df['Population'] / total_pop) * 100

# 3. The Metric: Representation Ratio
# This is the core metric: Influence (GDP) divided by Size (Pop)
df['Representation Ratio'] = df['% Global GDP'] / df['% Global Pop']

# 4. The Classifications

# A. CLEAN STATUS (For Colors & Tooltips)
def classify_clean(ratio):
    if ratio > 4.0: return 'Hyper-Represented'
    elif ratio >= 2.0: return 'Over-Represented'
    elif ratio >= 1.25: return 'Slight Surplus'
    elif ratio >= 0.8: return 'Proportional'
    elif ratio >= 0.5: return 'Slight Deficit'
    elif ratio >= 0.25: return 'Under-Represented'
    else: return 'Marginalized'

df['Representation Status'] = df['Representation Ratio'].apply(classify_clean)

# B. DETAILED STATUS (For Tooltips Only)
def classify_detailed(ratio):
    if ratio > 4.0: return 'Hyper-Represented (> 4.0)'
    elif ratio >= 2.0: return 'Over-Represented (2.0 - 4.0)'
    elif ratio >= 1.25: return 'Slight Surplus (1.25 - 2.0)'
    elif ratio >= 0.8: return 'Proportional (0.8 - 1.25)'
    elif ratio >= 0.5: return 'Slight Deficit (0.5 - 0.8)'
    elif ratio >= 0.25: return 'Under-Represented (0.25 - 0.5)'
    else: return 'Marginalized (< 0.25)'

df['Status_Tooltip'] = df['Representation Ratio'].apply(classify_detailed)

# C. CLICK LABELS (For Scatter Calculator)
def create_click_label(row):
    ratio = row['Representation Ratio']
    status = row['Representation Status']
    if ratio >= 1:
        return f"{ratio:.1f}x Proportional Share\n({status})"
    else:
        return f"{ratio:.2f}x Proportional Share\n({status})"

df['Click_Label'] = df.apply(create_click_label, axis=1)

# D. SMART FORMATTING (For Tooltips)
def smart_format(val):
    if val < 0.01: return f"{val:.4f}%"
    else: return f"{val:.2f}%"

df['Tooltip_Pop'] = df['% Global Pop'].apply(smart_format)
df['Tooltip_GDP'] = df['% Global GDP'].apply(smart_format)

# ---------------------------------------------------------
# DATA EXPORT & RELOAD
# ---------------------------------------------------------
# Saving the processed data so I can submit it alongside the report
export_filename = 'InfluenceOfNations_Data.csv'
df.to_csv(export_filename, index=False)
print(f"SUCCESS: Processed data saved to '{export_filename}'")

# Reloading from the new file to ensure the dashboard uses the final submission data
df = pd.read_csv(export_filename)
print("SUCCESS: Data reloaded for verification.")

# Final verify
print("-" * 30)
print(df[['Name', 'Representation Status', 'Status_Tooltip']].head(5))

SUCCESS: Processed data saved to 'InfluenceOfNations_Data.csv'
SUCCESS: Data reloaded for verification.
------------------------------
          Name Representation Status                  Status_Tooltip
0  Afghanistan          Marginalized           Marginalized (< 0.25)
1      Albania          Proportional       Proportional (0.8 - 1.25)
2      Algeria     Under-Represented  Under-Represented (0.25 - 0.5)
3       Angola          Marginalized           Marginalized (< 0.25)
4    Argentina          Proportional       Proportional (0.8 - 1.25)


### Cell 3: The "Power Ratio" Choropleth Map

In [504]:
# Cell 3: Building the Map

# Setting the order for the legend so it reads logically from low to high
color_domain = [
    'Marginalized', 'Under-Represented', 'Slight Deficit',
    'Proportional',
    'Slight Surplus', 'Over-Represented', 'Hyper-Represented'
]

# Using a sequential purple scale that pops against white
color_range = [
    '#c4bedc', '#a99fcb', '#8f83ba', '#7668a9', '#5d4e98', '#463587', '#301d76'
]

# This is the cool bit - turning the legend into a filter button
select_status = alt.selection_point(fields=['Representation Status'], bind='legend')

# Drawing the map layer
map_chart = alt.Chart(world_source).mark_geoshape(
    stroke='black', strokeWidth=0.2
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df, 'ID', ['Name', 'Representation Ratio', 'Representation Status', 'Status_Tooltip', 'Region']),
    default='Unknown'
).encode(
    # Mapping color to the clean names
    color=alt.Color(
        'Representation Status:N',
        scale=alt.Scale(domain=color_domain, range=color_range),
        legend=alt.Legend(
            title="Representation Status",
            # Floating the legend in the Pacific Ocean to save space
            orient='none',
            legendX=40, legendY=250,
            titleFontSize=14, titleFontWeight='bold',
            labelFontSize=12, symbolSize=200,
            direction='vertical', padding=10,
            fillColor='white', strokeColor='grey', cornerRadius=5
        )
    ),
    # Only showing the essentials on hover
    tooltip=[
        alt.Tooltip('Name:N', title='Country'),
        alt.Tooltip('Status_Tooltip:N', title='Status') # Detailed version here
    ]
).project(
    type='equalEarth'
).properties(
    width=900, height=430, title='The Global Split'
).add_params(
    select_status
)

### Cell 4: The "Mechanics" Scatter Plot

In [505]:
# Cell 4: Building the Calculator (Scatter Plot)

# Tiny countries (like Tuvalu) show up as 0.00% unless I force 4 decimal places
def smart_format(val):
    if val < 0.01: return f"{val:.4f}%"
    else: return f"{val:.2f}%"

df['Tooltip_Pop'] = df['% Global Pop'].apply(smart_format)
df['Tooltip_GDP'] = df['% Global GDP'].apply(smart_format)

# Building the dynamic text label that appears when you click a dot
def create_click_label(row):
    ratio = row['Representation Ratio']
    status = row['Representation Status']
    if ratio >= 1:
        return f"{ratio:.1f}x Proportional Share\n({status})"
    else:
        return f"{ratio:.2f}x Proportional Share\n({status})"

df['Click_Label'] = df.apply(create_click_label, axis=1)

# Setting up the click interactions
select_highlight = alt.selection_point(fields=['Name'], on='click', empty=True)
select_calculator = alt.selection_point(fields=['Name'], on='click', empty=False)

# Log scales are needed because the difference between USA and Tonga is massive
scale_type = 'log'
domain_range = [0.0001, 100]
grid_values = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]

# Adding background text labels to explain the quadrants
label_data_1 = pd.DataFrame({'x': [0.005], 'y': [1], 'text': ['AMPLIFIED VOICE\n(Over-Represented)']})
label_top_left = alt.Chart(label_data_1).mark_text(
    align='left', fontSize=14, fontWeight='bold', opacity=0.35, color='purple', lineBreak='\n'
).encode(
    x=alt.X('x', scale=alt.Scale(type='log', domain=domain_range)),
    y=alt.Y('y', scale=alt.Scale(type='log', domain=domain_range)),
    text='text'
)

label_data_2 = pd.DataFrame({'x': [5], 'y': [0.005], 'text': ['DIMINISHED VOICE\n(Under-Represented)']})
label_bottom_right = alt.Chart(label_data_2).mark_text(
    align='right', fontSize=14, fontWeight='bold', opacity=0.35, color='purple', lineBreak='\n'
).encode(
    x=alt.X('x', scale=alt.Scale(type='log', domain=domain_range)),
    y=alt.Y('y', scale=alt.Scale(type='log', domain=domain_range)),
    text='text'
)

# The dashed line represents perfect equality (x=y)
line_data = pd.DataFrame({'x': [0.0001, 100], 'y': [0.0001, 100]})
diagonal_line = alt.Chart(line_data).mark_line(
    color='grey', strokeDash=[5, 5], opacity=0.5
).encode(
    x=alt.X('x', scale=alt.Scale(type=scale_type, domain=domain_range)),
    y=alt.Y('y', scale=alt.Scale(type=scale_type, domain=domain_range))
)

# Drawing the actual country dots
scatter_points = alt.Chart(df).mark_circle(size=100, stroke='black').encode(
    x=alt.X('% Global Pop',
            scale=alt.Scale(type=scale_type, domain=domain_range),
            axis=alt.Axis(values=grid_values),
            title='Global Population Share (%)'),

    y=alt.Y('% Global GDP',
            scale=alt.Scale(type=scale_type, domain=domain_range),
            axis=alt.Axis(values=grid_values),
            title='Global Influence Share (GDP %)'),

    color=alt.Color('Representation Status:N', scale=alt.Scale(domain=color_domain, range=color_range), legend=None),

    # Fading out dots that aren't selected
    opacity=alt.condition(select_highlight, alt.value(1.0), alt.value(0.3)),
    strokeWidth=alt.condition(select_calculator, alt.value(2), alt.value(0)),

    # Using the smart tooltips I made earlier
    tooltip=[
        alt.Tooltip('Name', title='Country'),
        alt.Tooltip('Status_Tooltip', title='Status'),
        alt.Tooltip('Representation Ratio', format='.2f', title='Representation Ratio'),
        alt.Tooltip('Tooltip_Pop', title='% Pop Share'),
        alt.Tooltip('Tooltip_GDP', title='% GDP Share')
    ]
).add_params(select_highlight, select_calculator)

# The vertical line that drops down when you click
gap_rule = alt.Chart(df).transform_filter(select_calculator).mark_rule(
    color='black', strokeWidth=2, strokeDash=[4, 4]
).encode(x='% Global Pop', y='% Global GDP', y2='% Global Pop')

# The text label attached to that line
gap_text = alt.Chart(df).transform_filter(select_calculator).mark_text(
    align='left', dx=10, dy=-10, fontSize=12, fontWeight='bold', color='black', lineBreak='\n'
).encode(
    x='% Global Pop',
    y='% Global GDP',
    text='Click_Label'
)

# Stacking all the layers together
final_scatter = (label_top_left + label_bottom_right + diagonal_line + scatter_points + gap_rule + gap_text).properties(
    width=700,
    height=430,
    title='The Representation Ratio Calculator'
)

final_scatter

alt.LayerChart(...)

### Cell 5: View 3 - The "Imbalance" Diverging Bar Chart

In [506]:
# Cell 5: Building the Regional Bar Chart

# Since I need to show regions, I have to re-aggregate the data here
df_region = df.groupby('Region')[['GDP', 'Population']].sum().reset_index()

# Re-calculating the percentages for these regional blocks
df_region['% Global GDP'] = (df_region['GDP'] / total_gdp) * 100
df_region['% Global Pop'] = (df_region['Population'] / total_pop) * 100

# Getting the ratio and status for the whole region
df_region['Representation Ratio'] = df_region['% Global GDP'] / df_region['% Global Pop']
df_region['Representation Status'] = df_region['Representation Ratio'].apply(classify_clean)
df_region['Status_Tooltip'] = df_region['Representation Ratio'].apply(classify_detailed)

# Flipping GDP to negative so it plots to the left of the axis
df_region['GDP_Plot'] = -df_region['% Global GDP']

# Setting up the region filter
select_region = alt.selection_point(fields=['Region'])
shared_x_scale = alt.Scale(domain=[-100, 100])

# Just a base layer to save typing
base = alt.Chart(df_region).encode(
    y=alt.Y('Region:N', axis=None),
    opacity=alt.condition(select_region, alt.value(1), alt.value(0.3))
)

# Defining a single tooltip list so both sides of the bar show the same info
unified_tooltip = [
    alt.Tooltip('Region'),
    alt.Tooltip('% Global GDP', format='.1f', title='% GDP Share'),
    alt.Tooltip('% Global Pop', format='.1f', title='% Pop Share'),
    alt.Tooltip('Representation Ratio', format='.2f'),
    alt.Tooltip('Status_Tooltip', title='Status')
]

# The Left Bars (Influence/GDP)
left_bars = base.mark_bar(stroke='black', strokeWidth=1).encode(
    x=alt.X('GDP_Plot:Q', scale=shared_x_scale, title=None, axis=alt.Axis(format='d', labelExpr="abs(datum.value)")),
    color=alt.Color('Representation Status:N', scale=alt.Scale(domain=color_domain, range=color_range), legend=None),
    tooltip=unified_tooltip
)

# The Right Bars (Population)
right_bars = base.mark_bar(stroke='black', strokeWidth=1).encode(
    x=alt.X('% Global Pop:Q', scale=shared_x_scale, title=None),
    color=alt.Color('Representation Status:N', scale=alt.Scale(domain=color_domain, range=color_range), legend=None),
    tooltip=unified_tooltip
)

# Putting the numbers at the ends of the bars
left_text = base.mark_text(align='right', dx=-5).encode(x='GDP_Plot:Q', text=alt.Text('% Global GDP', format='.1f'))
right_text = base.mark_text(align='left', dx=5).encode(x='% Global Pop:Q', text=alt.Text('% Global Pop', format='.1f'))

# Putting the region names smack in the middle
middle_labels = base.mark_text(align='center').encode(y=alt.Y('Region:N', axis=None), x=alt.value(0), text='Region')

# Custom titles below the axis
title_data = pd.DataFrame({
    'x': [-50, 50],
    'label': ['Global Influence Share (GDP %)', 'Global Population Share (%)']
})
custom_titles = alt.Chart(title_data).mark_text(dy=170, fontSize=10, fontWeight='bold').encode(x=alt.X('x', scale=shared_x_scale), text='label')

# Merging it all
regional_chart = (left_bars + left_text + right_bars + right_text + middle_labels + custom_titles).properties(
    width=1700,
    height=280,
    title=alt.TitleParams(
        text='The Regional Split',
        subtitle=[
            "Visualizing how the Global Pie is divided. Ideally, a region's Influence Share (Left) should roughly match its Population Share (Right).",
            "CAUTION: Regional averages often hide extreme outliers. Click a bar to filter the dashboard and reveal which specific nations are skewing the results."
        ],
        fontSize=15,
        subtitleFontSize=12,
        subtitleColor='#525252',
        anchor='middle'
    )
).add_params(select_region)

regional_chart

alt.LayerChart(...)

In [507]:
# Cell 6: Assembling the final dashboard

# ---------------------------------------------------------
# STEP 1: LINKING EVERYTHING TOGETHER
# ---------------------------------------------------------

# Updating the Map: It needs to listen to the region filter, the highlighter, AND the legend
final_map_linked = map_chart.transform_filter(
    select_region
).encode(
    # Logic: Show if (Country is clicked OR nothing clicked) AND (Status matches Legend OR nothing clicked)
    opacity=alt.condition(select_highlight & select_status, alt.value(1), alt.value(0.1)),

    strokeWidth=alt.condition(select_calculator, alt.value(2), alt.value(0.2)),
    stroke=alt.condition(select_calculator, alt.value('black'), alt.value('transparent'))
).add_params(
    select_highlight,
    select_calculator
)

# Updating the Scatter: Needs the exact same logic as the map
scatter_filtered = scatter_points.encode(
    opacity=alt.condition(select_highlight & select_status, alt.value(1.0), alt.value(0.1))
).transform_filter(select_region)

gap_rule_filtered = gap_rule.transform_filter(select_region)
gap_text_filtered = gap_text.transform_filter(select_region)

# Rebuilding the scatter with all the filters applied
final_scatter_linked = (label_top_left + label_bottom_right + diagonal_line + scatter_filtered + gap_rule_filtered + gap_text_filtered).properties(
    width=700,
    height=450,
    title=alt.TitleParams(
        text='The Representation Ratio Calculator',
        subtitle=[
            "Visualizing how each country's ratio is derived. The dotted line represents a ratio of 1.0 (Perfect Equality).",
            "The distance from this line defines how over- or under-represented a nation is in the global market."
        ],
        fontSize=15,
        subtitleFontSize=12,
        subtitleColor='#525252',
    )
)

# ---------------------------------------------------------
# STEP 2: ARRANGING THE LAYOUT
# ---------------------------------------------------------
# Putting the Map and Scatter side-by-side
top_row = alt.hconcat(final_map_linked, final_scatter_linked).resolve_scale(
    color='shared'
)

# Stacking the Regional chart underneath
dashboard = alt.vconcat(top_row, regional_chart, center=True).properties(
    title=alt.TitleParams(
        text='The Influence of Nations in the Global Market: Equal People, Unequal Power',
        subtitle=[
            "In the global market, GDP is Influence. A nation's share of world GDP determines the volume of its voice.",
            "Ideally, this voice would be proportional to its share of the population ('Equal Influence for Every Person').",
            "This dashboard audits the gap using the Representation Ratio (GDP Share ÷ Population Share). By measuring deviations from perfect equality, we reveal the truth:",
            "Who is disproportionately loud, and who is systematically silenced?",
            "",
            "INSTRUCTIONS: Click a Region (Bottom) to filter, a Country (Map) to audit, or a Legend Item (Left) to highlight a group."
        ],
        fontSize=32,
        subtitleFontSize=16,
        subtitleColor='#525252',
        anchor='start',
        offset=0
    )
).configure_view(stroke=None).configure_concat(spacing=10)

# ---------------------------------------------------------
# STEP 3: OUTPUT
# ---------------------------------------------------------
dashboard.save('InfluenceOfNations.html')

print("SUCCESS: Final Dashboard Generated with updated subtitle.")
dashboard

C:\Users\Cameron\OneDrive\Desktop\Stage 3\VisualizingEconomicCountryData\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3699: UserWarning: Automatically deduplicated selection parameter with identical configuration. If you want independent parameters, explicitly name them differently (e.g., name='param1', name='param2'). See https://github.com/vega/altair/issues/3891
  exec(code_obj, self.user_global_ns, self.user_ns)


SUCCESS: Final Dashboard Generated with updated subtitle.


alt.VConcatChart(...)